In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tqdm

In [2]:
%matplotlib inline

In [3]:
def mostrar_imagenes(image_array, num_images=25, size=(28, 28)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in a uniform grid.
    '''
    image_plot = image_array.reshape(-1, *size) * 255
    image_plot = image_plot.astype(np.uint8)
#     plt.figure(figsize=(15, 4.5))
    for i in range(num_images):
        plt.subplot(5, int(num_images) / 5, i + 1)
        plt.imshow(image_plot[i].astype(np.uint8), cmap=plt.cm.binary)
        plt.axis('off')
    plt.show()

In [4]:
def obtener_bloque_generador(dim_entrada: int, dim_salida: int) -> tf.keras.Model:
    ''' Genera una capa lineal con normalización por bloques '''
    entrada = tf.keras.Input(shape=dim_entrada)
    capa_lineal = tf.keras.layers.Dense(units=dim_salida)(entrada)
    normalizacion = tf.keras.layers.BatchNormalization()(capa_lineal)
    salida_relu = tf.keras.activations.relu(normalizacion)
    return tf.keras.Model(inputs=entrada, outputs=salida_relu)

def obtener_generador(dim_z: int = 10, dim_imagen: int = 784, dim_oculta: int = 128) -> tf.keras.Model:
    ''' Generador de imágenes '''
    entrada = tf.keras.Input(shape=dim_z)
    salida_1 = obtener_bloque_generador(dim_entrada=dim_z, dim_salida=dim_oculta)(entrada)
    salida_2 = obtener_bloque_generador(dim_entrada=dim_oculta, dim_salida=dim_oculta*2)(salida_1)
    salida_3 = obtener_bloque_generador(dim_entrada=dim_oculta*2, dim_salida=dim_oculta*4)(salida_2)
    salida_4 = obtener_bloque_generador(dim_entrada=dim_oculta*4, dim_salida=dim_oculta*8)(salida_3)
    salida_lineal = tf.keras.layers.Dense(units=dim_imagen)(salida_4)
    salida_sigmoide = tf.keras.activations.sigmoid(salida_lineal)
    return tf.keras.Model(inputs=entrada, outputs=salida_sigmoide)

In [5]:
def create_generator():
    model = tf.keras.Sequential()
    
    # creating Dense layer with units 7*7*256(batch_size) and input_shape of (100,)
    model.add(tf.keras.layers.Dense(7*7*256, use_bias=False, input_shape=(64,)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Reshape((7, 7, 256)))

    model.add(tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

In [6]:
obtener_generador().summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
model (Functional)           (None, 128)               1920      
_________________________________________________________________
model_1 (Functional)         (None, 256)               34048     
_________________________________________________________________
model_2 (Functional)         (None, 512)               133632    
_________________________________________________________________
model_3 (Functional)         (None, 1024)              529408    
_________________________________________________________________
dense_4 (Dense)              (None, 784)               803600    
_________________________________________________________________
tf.math.sigmoid (TFOpLambda) (None, 784)               0   

In [7]:
def obtener_ruido(numero_muestra: int, dim_z: int) -> tf.random.normal:
    ''' Genera vector aleatorios de dimension (numero_muestra, dim_z) '''
    return tf.random.normal((numero_muestra, dim_z))

In [8]:
def obtener_bloque_discriminador(dim_entrada: int, dim_salida: int) -> tf.keras.Model:
    ''' Genera una capa lineal con normalización por bloques '''
    entrada = tf.keras.Input(shape=dim_entrada)
    capa_lineal = tf.keras.layers.Dense(units=dim_salida)(entrada)
    salida_leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)(capa_lineal)
    return tf.keras.Model(inputs=entrada, outputs=salida_leaky_relu)

def obtener_discriminador(dim_imagen: int = 784, dim_oculta: int = 128) -> tf.keras.Model:
    ''' Generador de imágenes '''
    entrada = tf.keras.Input(shape=dim_imagen)
    salida_1 = obtener_bloque_discriminador(dim_entrada=dim_imagen, dim_salida=dim_oculta*4)(entrada)
    salida_2 = obtener_bloque_discriminador(dim_entrada=dim_oculta*4, dim_salida=dim_oculta*2)(salida_1)
    salida_3 = obtener_bloque_discriminador(dim_entrada=dim_oculta*2, dim_salida=dim_oculta)(salida_2)
    salida_lineal = tf.keras.layers.Dense(units=1)(salida_3)
    salida_sigmoide = tf.keras.activations.sigmoid(salida_lineal)
    return tf.keras.Model(inputs=entrada, outputs=salida_sigmoide)

In [9]:
def create_discriminator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))

    return model

In [10]:
obtener_discriminador().summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
model_5 (Functional)         (None, 512)               403968    
_________________________________________________________________
model_6 (Functional)         (None, 256)               132352    
_________________________________________________________________
model_7 (Functional)         (None, 128)               33408     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
_________________________________________________________________
tf.math.sigmoid_1 (TFOpLambd (None, 1)                 0         
Total params: 569,857
Trainable params: 568,065
Non-trainable params: 1,792
_________________________________________________

In [11]:
def obtener_perdida_discriminador(generador: tf.keras.Model, discriminador: tf.keras.Model, imagenes_real, num_imagenes, dim_z):
    ''' Obtiene perdida del discriminador '''
    ruido = obtener_ruido(numero_muestra=num_imagenes, dim_z=dim_z)
    imagenes_falsa = generador(ruido)
    prediccion_falsa = discriminador(tf.stop_gradient(imagenes_falsa))
    target_falsa = tf.zeros_like(prediccion_falsa)
#     perdida_falsa = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=target_falsa, logits=prediccion_falsa))
    perdida_falsa = tf.reduce_mean(tf.keras.losses.binary_crossentropy(y_true=target_falsa, y_pred=prediccion_falsa))
    
    prediccion_real = discriminador(imagenes_real)
    target_real = tf.ones_like(prediccion_real)
#     perdida_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=target_real, logits=prediccion_real))
    perdida_real = tf.reduce_mean(tf.keras.losses.binary_crossentropy(y_true=target_real, y_pred=prediccion_real))

    perdida_discriminador = (perdida_falsa + perdida_real) / 2
    return perdida_discriminador

In [12]:
def obtener_perdida_generador(generador, discriminador, num_imagenes, dim_z):
    ruido = obtener_ruido(numero_muestra=num_imagenes, dim_z=dim_z)
    imagenes_falsa = generador(ruido)
    prediccion_falsa = discriminador(imagenes_falsa)
    target_falsa = tf.ones_like(prediccion_falsa)
#     perdida_generador = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=target_falsa, logits=prediccion_falsa))
    perdida_generador = tf.reduce_mean(tf.keras.losses.binary_crossentropy(y_true=target_falsa, y_pred=prediccion_falsa))

    return perdida_generador

In [13]:
def obtener_mnist_data(batch_size=128):
    (mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = tf.keras.datasets.mnist.load_data()
    mnist_x_train = mnist_x_train.reshape(-1, 784)
    mnist_x_test = mnist_x_test.reshape(-1, 784)
    mnist_data = np.concatenate((mnist_x_train, mnist_x_test), axis=0)
    mnist_data = mnist_data / 255.0
    mnist_data = mnist_data.reshape(-1, 28, 28, 1)
    mnist_target = np.concatenate((mnist_y_train, mnist_y_test), axis=0)
    dataset = tf.data.Dataset.from_tensor_slices((mnist_data, mnist_target))
    dataset = dataset.shuffle(len(mnist_data)).batch(batch_size)
    return dataset

In [14]:
dataset = obtener_mnist_data(128)

In [15]:
# generador = obtener_generador(dim_z=64)
generador = create_generator()
generador_optimizador = tf.keras.optimizers.Adam(learning_rate=1e-4)

# discriminador = obtener_discriminador()
discriminador = create_discriminator()
discriminador_optimizador = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [16]:
dim_z = 64
n_epochs = 2000

In [ ]:
paso_actual = 0
media_perdida_generador = 0
media_perdida_discriminador = 0

# for epoch in tqdm.tqdm(range(n_epochs)):
for epoch in range(n_epochs):
    
#     for real, _ in dataset:
    for real, _ in tqdm.tqdm(dataset):
        actual_tamanho_batch = len(real)
        
        with tf.GradientTape() as tape:
            perdida_discriminador = obtener_perdida_discriminador(generador, discriminador, real, actual_tamanho_batch, dim_z)
            gradiente_discriminador = tape.gradient(perdida_discriminador, discriminador.trainable_variables)
            discriminador_optimizador.apply_gradients(zip(gradiente_discriminador, discriminador.trainable_variables))
        
        media_perdida_discriminador += perdida_discriminador / actual_tamanho_batch
            
        with tf.GradientTape() as tape:
            perdida_generador = obtener_perdida_generador(generador, discriminador, actual_tamanho_batch, dim_z)
            gradiente_generador = tape.gradient(perdida_generador, generador.trainable_variables)
            generador_optimizador.apply_gradients(zip(gradiente_generador, generador.trainable_variables))
        
        media_perdida_generador += perdida_generador / actual_tamanho_batch
        
    if epoch % 1 == 0:
        print(f"Epoch: {epoch}, discriminador: {media_perdida_discriminador}, generador: {media_perdida_generador}")
        
        mostrar_imagenes(generador(obtener_ruido(actual_tamanho_batch, dim_z)).numpy(), num_images=25, size=(28, 28))
        mostrar_imagenes(real.numpy(), num_images=25, size=(28, 28))
    
    media_perdida_generador = 0
    media_perdida_discriminador = 0

 98%|██████████████████████████████████████████████████████████████████████████████▏ | 535/547 [11:04<00:15,  1.32s/it]

In [ ]:
real.numpy().mean()

In [ ]:
generador(obtener_ruido(actual_tamanho_batch, dim_z)).numpy().max()